<a href="https://colab.research.google.com/github/zhanmdd/glassdoor-reviews-sentiment-BERT/blob/master/Glassdoor_Reviews_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries and Uploading the Dataset


In [ ]:
import pandas as pd
from google.colab import files
import io

In [ ]:
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['data.csv']))

Saving data.csv to data.csv


#What needs to be 'cleaned' and 'fixed'?

###Observing the dataset

In [ ]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10124 entries, 0 to 10123
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             10124 non-null  object
 1   employee_title   5926 non-null   object
 2   location         6239 non-null   object
 3   employee_status  10124 non-null  object
 4   review_title     10113 non-null  object
 5   main_text        10124 non-null  object
 6   pros             10124 non-null  object
 7   cons             10124 non-null  object
 8   rating_overall   10124 non-null  object
 9   company_name     10124 non-null  object
dtypes: object(10)
memory usage: 791.1+ KB


,date,employee_title,location,employee_status,review_title,main_text,pros,cons,rating_overall,company_name
0,Wed Jan 22 2020 08:38:01 GMT+0900 (Korean Stan...,Market Research Analyst II,"Richmond, VA",Current Employee,Best company I had ever worked for in 3 decades,I have been working at CoStar Group full-time ...,Senior Management is always cheerful and knows...,We have to share our building.,5.0\n★★★★★,CoStar-Group
1,Thu Jul 16 2020 10:23:06 GMT+0900 (Korean Stan...,Inside Sales Representative,NaN,Current Employee,Great Place to Make Money,I have been working at CoStar Group full-time,The product is the best out and there and it s...,Lack of communication between departments,5.0\n★★★★★,CoStar-Group
2,Thu Jul 16 2020 10:49:00 GMT+0900 (Korean Stan...,Inside Sales Representative,"Washington, DC",Current Employee,Great place to start a career and make money,I have been working at CoStar Group full-time ...,The inside sales team is an entry level sales ...,Communication from departments is lacking due ...,5.0\n★★★★★,CoStar-Group
3,Wed Jul 15 2020 11:13:00 GMT+0900 (Korean Stan...,Senior Account Manager,"Washington, DC",Former Employee,Great Company,I worked at CoStar Group full-time for more th...,Experience. Growth. Culture. CEO. Motivating,Inexperienced managers who managed in demotiva...,4.0\n★★★★★,CoStar-Group
4,Tue Jul 21 2020 17:49:46 GMT+0900 (Korean Stan...,Tenant Research Associate,"Richmond, VA",Current Employee,DO NOT GET SUCKED FOR THE MONEY!!!,I have been working at CoStar Group full-time ...,Pay good benefits coworkers Christmas party gr...,"They hate to be the word ""call-center,"" but th...",2.0\n★★★★★,CoStar-Group


###List of things to be done

1.   Convert **'date'** column into datetime type for EDA
2.   Parse **'location'** column into **'city'** and **'state'** columns for EDA
3. Combine **'review_title',	'main_text',	'pros',	'cons'** into a single **'reviews'** column for a model
4. Remove '★★★★★' in **'rating_overall'** and convert it to float for EDA and a model
5. Include 'sentiment' column
6. Improve the order of the columns







#Data Cleaning

###1.   Converting **'date'** column into datetime

In [ ]:
#loop through each row and select [0:24] characters from the string:
#Example: Wed Jan 22 2020 08:38:01
for x in range(0, len(df)):
  df.date[x] = df.date[x][0:24]
df.date = pd.to_datetime(df.date)
df.head()

,date,employee_title,location,employee_status,review_title,main_text,pros,cons,rating_overall,company_name
0,2020-01-22 08:38:01,Market Research Analyst II,"Richmond, VA",Current Employee,Best company I had ever worked for in 3 decades,I have been working at CoStar Group full-time ...,Senior Management is always cheerful and knows...,We have to share our building.,5.0\n★★★★★,CoStar-Group
1,2020-07-16 10:23:06,Inside Sales Representative,NaN,Current Employee,Great Place to Make Money,I have been working at CoStar Group full-time,The product is the best out and there and it s...,Lack of communication between departments,5.0\n★★★★★,CoStar-Group
2,2020-07-16 10:49:00,Inside Sales Representative,"Washington, DC",Current Employee,Great place to start a career and make money,I have been working at CoStar Group full-time ...,The inside sales team is an entry level sales ...,Communication from departments is lacking due ...,5.0\n★★★★★,CoStar-Group
3,2020-07-15 11:13:00,Senior Account Manager,"Washington, DC",Former Employee,Great Company,I worked at CoStar Group full-time for more th...,Experience. Growth. Culture. CEO. Motivating,Inexperienced managers who managed in demotiva...,4.0\n★★★★★,CoStar-Group
4,2020-07-21 17:49:46,Tenant Research Associate,"Richmond, VA",Current Employee,DO NOT GET SUCKED FOR THE MONEY!!!,I have been working at CoStar Group full-time ...,Pay good benefits coworkers Christmas party gr...,"They hate to be the word ""call-center,"" but th...",2.0\n★★★★★,CoStar-Group


###2.   Parsing **'location'** column into **'city'** and **'state'** columns

In [ ]:
#new empty columns
df['city'] = ''
df['state'] = ''

#make sure all values are str type
df['location'] = df['location'].astype(str)

#loop through each value
for x in range(0, len(df)):
  l = len(df['location'][x].split(', ')) #split each value: City, State into list [City, State]
  if  l >= 2:
    df['city'][x] = df['location'][x].split(', ')[0] #insert City under 'city' column
    df['state'][x] = df['location'][x].split(', ')[1] #insert State under 'state' column
  else:
    pass

df = df.drop(columns = ['location']) #remove the initial 'location' column
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,date,employee_title,employee_status,review_title,main_text,pros,cons,rating_overall,company_name,city,state
0,2020-01-22 08:38:01,Market Research Analyst II,Current Employee,Best company I had ever worked for in 3 decades,I have been working at CoStar Group full-time ...,Senior Management is always cheerful and knows...,We have to share our building.,5.0\n★★★★★,CoStar-Group,Richmond,VA
1,2020-07-16 10:23:06,Inside Sales Representative,Current Employee,Great Place to Make Money,I have been working at CoStar Group full-time,The product is the best out and there and it s...,Lack of communication between departments,5.0\n★★★★★,CoStar-Group,,
2,2020-07-16 10:49:00,Inside Sales Representative,Current Employee,Great place to start a career and make money,I have been working at CoStar Group full-time ...,The inside sales team is an entry level sales ...,Communication from departments is lacking due ...,5.0\n★★★★★,CoStar-Group,Washington,DC
3,2020-07-15 11:13:00,Senior Account Manager,Former Employee,Great Company,I worked at CoStar Group full-time for more th...,Experience. Growth. Culture. CEO. Motivating,Inexperienced managers who managed in demotiva...,4.0\n★★★★★,CoStar-Group,Washington,DC
4,2020-07-21 17:49:46,Tenant Research Associate,Current Employee,DO NOT GET SUCKED FOR THE MONEY!!!,I have been working at CoStar Group full-time ...,Pay good benefits coworkers Christmas party gr...,"They hate to be the word ""call-center,"" but th...",2.0\n★★★★★,CoStar-Group,Richmond,VA


### 3. Combining 'review_title', 'main_text', 'pros', 'cons' into a single 'review' column for a model

In [ ]:
#make sure that these columns are str type
df[['review_title', 'main_text', 'pros', 'cons']] = df[['review_title', 'main_text', 'pros', 'cons']].astype(str)

#This funtion writes a dot at the end of a sentence if it doesn't end with one
#We need it when combining the columns to have separations between sentences.
def period(string):
  if string[-1] == '.' or string[-1] == '!' or string[-1] == '?':
    return string
  else:
    new_string = string = string +'. '
    return new_string 

for i in range(0,len(df)):
  df.review_title[i] = period(df.review_title[i])
  df.main_text[i] = period(df.main_text[i])
  df.pros[i] = period(df.pros[i])
  df.cons[i] = period(df.cons[i])

#combine 'review_title', 'main_text', 'pros', 'cons' into single 'review' column
df['review'] = df['review_title'] + df['main_text'] + df['pros'] + df['cons']
df = df.drop(columns = ['review_title','main_text','pros','cons'])

df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/i

,date,employee_title,employee_status,rating_overall,company_name,city,state,review
0,2020-01-22 08:38:01,Market Research Analyst II,Current Employee,5.0\n★★★★★,CoStar-Group,Richmond,VA,Best company I had ever worked for in 3 decade...
1,2020-07-16 10:23:06,Inside Sales Representative,Current Employee,5.0\n★★★★★,CoStar-Group,,,Great Place to Make Money. I have been working...
2,2020-07-16 10:49:00,Inside Sales Representative,Current Employee,5.0\n★★★★★,CoStar-Group,Washington,DC,Great place to start a career and make money. ...
3,2020-07-15 11:13:00,Senior Account Manager,Former Employee,4.0\n★★★★★,CoStar-Group,Washington,DC,Great Company. I worked at CoStar Group full-t...
4,2020-07-21 17:49:46,Tenant Research Associate,Current Employee,2.0\n★★★★★,CoStar-Group,Richmond,VA,DO NOT GET SUCKED FOR THE MONEY!!!I have been ...


### 4. Removing '★★★★★' in 'rating_overall' and converting it to float

In [ ]:
df['rating'] = pd.to_numeric(df.rating_overall.str.replace('★★★★★', '').str.strip())
df.drop(columns = ['rating_overall'])

,date,employee_title,employee_status,company_name,city,state,review,rating
0,2020-01-22 08:38:01,Market Research Analyst II,Current Employee,CoStar-Group,Richmond,VA,Best company I had ever worked for in 3 decade...,5.0
1,2020-07-16 10:23:06,Inside Sales Representative,Current Employee,CoStar-Group,,,Great Place to Make Money. I have been working...,5.0
2,2020-07-16 10:49:00,Inside Sales Representative,Current Employee,CoStar-Group,Washington,DC,Great place to start a career and make money. ...,5.0
3,2020-07-15 11:13:00,Senior Account Manager,Former Employee,CoStar-Group,Washington,DC,Great Company. I worked at CoStar Group full-t...,4.0
4,2020-07-21 17:49:46,Tenant Research Associate,Current Employee,CoStar-Group,Richmond,VA,DO NOT GET SUCKED FOR THE MONEY!!!I have been ...,2.0
...,...,...,...,...,...,...,...,...
10119,2014-12-09 21:49:42,NaN,,Perficient,,,Growing Company. I have been working at Perfic...,4.0
10120,2014-12-03 12:29:17,NaN,,Perficient,,,Solid consulting company. I have been working ...,4.0
10121,2014-11-26 11:29:18,Software Developer,Current Employee,Perficient,San Francisco,CA,"java, html, css developer. I have been working...",3.0
10122,2014-11-06 11:23:58,NaN,Current Employee,Perficient,,,Overall good company. I have been working at P...,4.0


### 5. Including 'sentiment' column

In [ ]:
def create_sentiment(x):
  if x <= 2:
    return 0
  elif x == 3:
    return 1
  else: 
    return 2
df['sentiment'] = df.rating.apply(create_sentiment)
df.head()

,date,employee_title,employee_status,rating_overall,company_name,city,state,review,rating,sentiment
0,2020-01-22 08:38:01,Market Research Analyst II,Current Employee,5.0\n★★★★★,CoStar-Group,Richmond,VA,Best company I had ever worked for in 3 decade...,5.0,2
1,2020-07-16 10:23:06,Inside Sales Representative,Current Employee,5.0\n★★★★★,CoStar-Group,,,Great Place to Make Money. I have been working...,5.0,2
2,2020-07-16 10:49:00,Inside Sales Representative,Current Employee,5.0\n★★★★★,CoStar-Group,Washington,DC,Great place to start a career and make money. ...,5.0,2
3,2020-07-15 11:13:00,Senior Account Manager,Former Employee,4.0\n★★★★★,CoStar-Group,Washington,DC,Great Company. I worked at CoStar Group full-t...,4.0,2
4,2020-07-21 17:49:46,Tenant Research Associate,Current Employee,2.0\n★★★★★,CoStar-Group,Richmond,VA,DO NOT GET SUCKED FOR THE MONEY!!!I have been ...,2.0,0


###6. Improving the order of the columns

In [ ]:
df = df[['company_name', 'employee_title', 'employee_status', 'state', 'city', 'date', 'rating', 'sentiment', 'review']]

In [ ]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10124 entries, 0 to 10123
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   company_name     10124 non-null  object        
 1   employee_title   5926 non-null   object        
 2   employee_status  10124 non-null  object        
 3   state            10124 non-null  object        
 4   city             10124 non-null  object        
 5   date             10124 non-null  datetime64[ns]
 6   rating           10124 non-null  float64       
 7   sentiment        10124 non-null  int64         
 8   review           10124 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 712.0+ KB


,company_name,employee_title,employee_status,state,city,date,rating,sentiment,review
0,CoStar-Group,Market Research Analyst II,Current Employee,VA,Richmond,2020-01-22 08:38:01,5.0,2,Best company I had ever worked for in 3 decade...
1,CoStar-Group,Inside Sales Representative,Current Employee,,,2020-07-16 10:23:06,5.0,2,Great Place to Make Money. I have been working...
2,CoStar-Group,Inside Sales Representative,Current Employee,DC,Washington,2020-07-16 10:49:00,5.0,2,Great place to start a career and make money. ...
3,CoStar-Group,Senior Account Manager,Former Employee,DC,Washington,2020-07-15 11:13:00,4.0,2,Great Company. I worked at CoStar Group full-t...
4,CoStar-Group,Tenant Research Associate,Current Employee,VA,Richmond,2020-07-21 17:49:46,2.0,0,DO NOT GET SUCKED FOR THE MONEY!!!I have been ...


#Exporting

In [ ]:
from google.colab import files
df.to_csv('df_cleaned.csv', index=False) 
files.download('df_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>